<a href="https://colab.research.google.com/github/Simaregele/TrainProject/blob/master/CianHH_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time


In [0]:
# дата фрейм для хранения данных
df = pd.DataFrame() 


In [0]:
new_page = requests.get(url_list[0]) # а тут мы берем первый же ЮРЛ

In [0]:
soup = BeautifulSoup(new_page.text, 'html.parser') 

In [0]:
card = soup.find_all(class_='_93444fe79c--card--_yguQ')[0]
card_title = soup.find_all('a', class_="c6e8ba5398--header--1fV2A")


In [0]:
# Не все работает
def create_url(page_number):
  # создает URL страницы
  return "https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={}&region=1&room1=1&room9=1&type=4".format(str(page_number))

def get_navigation_last_num_element(element_number, url):
  # возвращает последний элемент нав меню внизу страницы
  new_page = requests.get(url)
  soup = BeautifulSoup(new_page.text, 'html.parser')
  return soup.find_all(class_="_93444fe79c--list-itemLink--3o7_6")[element_number].get_text()


def count_number_of_category_pages(url):
  # считает кол-во страниц в выбранной категории, запускается на странице 
  while True:
    pages_menu_last__numeric_element = get_navigation_last_num_element(-1, url)
    if pages_menu_last__numeric_element.isdigit():
      # проверяет какой последний элемент, если тот цифра то значит это индекс
      # последней страницы
      return pages_menu_last__numeric_element
    else:
      # в ином случае вытаскивает предпослежний элемент и использует в качестве
      # индекса для генерация новой страницы на которой проверяется последний
      # элемент
      # может быть 1 
      pages_menu_last__numeric_element = get_navigation_last_num_element(-2, url)
      url = create_url(pages_menu_last__numeric_element)
      inner_soup = BeautifulSoup(requests.get(url).text, 'html.parser') 



In [0]:
# генерим странички по квартирам: студия или однушка, аренда на длительный срок москва
# это список ЮРЛ страниц с карточками
url_list = []
for i in range(1, 54):
  url = create_url(i)
  print(url)
  url_list.append(url)

In [0]:
# функция по генерированию URl с карточками объявлений
DEAL_TYPE = 'rent' # sale - покупка, напишите в кавычках sale если хотите
  # посомтреть покупку (пока не работает функция)
PAGE_COUNT = 54 # надо ввести кол-во страниц сцществующих, на циане
def url_list_generator(length):
  url_list = []
  page_number = 0
  url = create_url(page_number)
  for i in range(1,length):
    url = create_url(page_number)
    url_list.append(url)
    page_number += 1
  return url_list



In [0]:
def create_soup(url):
  # делаем СУУП :))
  new_page = requests.get(url)
  soup = BeautifulSoup(new_page.text, 'html.parser') 
  return soup

def get_ads_from_categorial_page(soup):
  # извлекает карточки
  cards = soup.find_all(class_='_93444fe79c--card--_yguQ')
  return cards


In [0]:
# Берем ЮРЛ карточки
def get_ads_url(card):
  card_url = card.find('a', class_="c6e8ba5398--header--1fV2A", href=True)['href']
  return card_url

In [0]:
# надо сделать функции для каждой 

def get_title(card_page):
  # Наименование объявления тег H1
  if card_page.find('h1', class_="a10a3f92e9--title--2Widg"):
    return card_page.find('h1', class_="a10a3f92e9--title--2Widg").get_text()
  return "No title"

def get_adress(card_page):
  if card_page.find('address', class_= "a10a3f92e9--address--140Ec"):
    return card_page.find('address', class_= "a10a3f92e9--address--140Ec").get_text()
  return "No adress"

def get_metro_station():
  # список станций
  if card_page.find_all('li', class_="a10a3f92e9--underground--kONgx"):
    metro_station = card_page.find_all('li', class_="a10a3f92e9--underground--kONgx")
    list_of_stations = []
    for i in metro_station:
      list_of_stations.append(i.get_text())
    return list_of_stations
  return "No stations"

def apartments_info():
  if card_page.find_all('div', class_="a10a3f92e9--info-text--2uhvD"):
    partments_info = card_page.find_all('div', class_="a10a3f92e9--info-text--2uhvD")
    all_apartmenrs_info = []
    for q in apartments_info:
      all_apartmenrs_info.append(q.get_text())
    return all_apartmenrs_info
  return "No info"

def text_subscription():
  if card_page.find('p', class_="a10a3f92e9--description-text--1_Lup"):    
    return card_page.find('p', class_="a10a3f92e9--description-text--1_Lup").get_text()
  return "Wow"



def get_all_data_from_ad_page_and_save_to_df(card_page, df):
  # вытаскиваем значения из подающейся на вход card_page и сохраняеем все в DF,
  # в месте где добавляется время добавления объявления так-же закидывается
  # время парсинга инфы


  # дополнительная инфа, есть ли стиралк, микроволновка и т.д.
  if card_page.find('ul', class_="a10a3f92e9--container--L-EIV"):
    additional_appartments_info = card_page.find('ul', class_="a10a3f92e9--container--L-EIV")
    appartments_features = []
    for i in additional_appartments_info:
      appartments_features.append(i.get_text())
    df['appartments_features'] = [str(appartments_features)]

  # еще дополнительные фичи, тут вроде площадь жидая совмещенный ли санузел и т.д.
  additional_features = []
  for i in card_page.find_all(class_="a10a3f92e9--value--3Ftu5"):
    additional_features.append(i.get_text())
  df['additional_features'] = [str(additional_features)]

  # стоимость аренды
  rent_price = card_page.find(class_="a10a3f92e9--price_value--1iPpd").get_text().replace(u'\xa0', ' ')
  df['rent_price'] = [rent_price]
  # информация по стоимости аренды
  rent_price_info = card_page.find(class_="a10a3f92e9--more_price_rent---5hwY").get_text().replace(u'\xa0', ' ')
  df['rent_price_info'] = rent_price_info
  # дополнительная информация по оплае
  additional_payment_info = card_page.find(class_="a10a3f92e9--desc"
  "ription--2xRVn").get_text().replace(u'\xa0', ' ').split(",")
  df['additional_payment_info'] = [additional_payment_info]
  # номера телефонов!!
  phones = card_page.find(class_="a10a3f92e9--container--35iPF").get_text().replace('Показать телефон', '').replace('Написать сообщение', '').split('+')
  df['phones'] = [phones] 
  # название агентства
  if card_page.find('h2', class_="a10a3f92e9--title--2gUWg"):
    agency_name = card_page.find('h2', class_="a10a3f92e9--title--2gUWg").get_text()
    df['agency_name'] = [agency_name]
  elif card_page.find(class_="a10a3f92e9--id--LA2Ew"):
    owner_name = card_page.find(class_="a10a3f92e9--id--LA2Ew").get_text()
    df['owner_name'] = [owner_name]

  
  # дата загрузки объявления, кол-во просмотров
  import datetime
  upload_d_t = card_page.find(class_="a10a3f92e9--container--3nJ0d").get_text()
  view_statistics = card_page.find(class_="a10a3f92e9--link--1t8n1 a10a3f92e9--link--2mJJk").get_text()
  if "сегодня" in upload_d_t:
    date = str(datetime.datetime.today())
  elif 'вчера' in upload_d_t:
    date = str(datetime.datetime.today())
  upload_d_t = [date, upload_d_t]
  df['upload_d_t'] = [upload_d_t]
  df['view_statistics'] = [view_statistics]
  
  return df


In [0]:
def get_page_from_ad_url(ad_url):
  # переходим на страниwу и парсим все данные со страницы
  get_page = requests.get(ad_url)
  ad_page = BeautifulSoup(get_page.text, 'html.parser')
  return ad_page

In [0]:
df = pd.DataFrame() 

In [29]:
# парсим пробегаемся по сгенерированным страницам, собираем карточки, переходим на карточку, собираем
# данне с карточки сохраняем в DF добавляем к большому DF
page_list = url_list_generator(54)
for i in page_list:
  # у нас есть все юрл страничек, теперь для каждой страницы нам надо извлечь список карточек
  soup = create_soup(i)
  ads_url = get_ads_from_categorial_page(soup) # функция
  last_row = df.shape[0]
  for idx, val in enumerate(ads_url):
    # добавим сна, чтобы не блочил сервак
    time.sleep(0.09)
    ad_url = get_ads_url(val)
    ad_page = get_page_from_ad_url(ad_url)    
    index = last_row + idx
    print(index, ad_url)
    get_all_data_from_ad_page_and_save_to_df(ad_page, df)
    
  


0 https://www.cian.ru/rent/flat/224019372/
1 https://www.cian.ru/rent/flat/223605492/
2 https://www.cian.ru/rent/flat/224322197/
3 https://www.cian.ru/rent/flat/224079646/
4 https://www.cian.ru/rent/flat/223916683/
5 https://www.cian.ru/rent/flat/223916225/
6 https://www.cian.ru/rent/flat/196157919/
7 https://www.cian.ru/rent/flat/224266597/
8 https://www.cian.ru/rent/flat/223549505/
9 https://www.cian.ru/rent/flat/221662984/
10 https://www.cian.ru/rent/flat/190049857/
11 https://www.cian.ru/rent/flat/223721765/
12 https://www.cian.ru/rent/flat/222531150/
13 https://www.cian.ru/rent/flat/224213145/
14 https://www.cian.ru/rent/flat/224124691/
15 https://www.cian.ru/rent/flat/209528336/
16 https://www.cian.ru/rent/flat/223365281/
17 https://www.cian.ru/rent/flat/223317443/
18 https://www.cian.ru/rent/flat/224258430/
19 https://www.cian.ru/rent/flat/224176391/
20 https://www.cian.ru/rent/flat/224274262/
21 https://www.cian.ru/rent/flat/224291239/
22 https://www.cian.ru/rent/flat/224347306

TypeError: ignored

In [30]:
df

,title,adress,metro,all_apartmenrs_info,apartsments_text_subscription,appartments_features,additional_features,rent_price,rent_price_info,additional_payment_info,phones,agency_name,upload_d_t,view_statistics,owner_name
0,"1-комн. квартира, 70 м²","Москва, ЦАО, р-н Хамовники, ул. Плющиха, 37На ...","['Смоленская ⋅ 13 мин. пешком', 'Киевская ⋅ ...","[70 м², 50 м², 10 м², 5 из 5]",ID: 13518 БЕЗ КОМИССИИ ДЛЯ КЛИЕНТА!!! КОЛЛЕГАМ...,"['Холодильник', 'Посудомоечная машина', 'Стира...","['Апартаменты', '30 м²', '1 совмещенный', '1 б...",119 000 ₽/мес.,Комм. платежи включены (без счётчиков),"[Залог 119 000 ₽, без комиссии, предоплата з...","[, 7 966 300-78-39]",MONROYAL GLOBAL MANSION,"[2020-01-13 17:56:05.865112, сегодня, 14:20]","869 просмотров, 26 за сегодня",ID 47162865


In [0]:
#извлек название и адрес
title = card_page.find('h1', class_="a10a3f92e9--title--2Widg").get_text()
adress = card_page.find('address', class_= "a10a3f92e9--address--140Ec").get_text()
df['title'] = [title]
df['adress'] = [adress] 

In [0]:
# станции метро

metro_station = card_page.find_all('li', class_="a10a3f92e9--underground--kONgx")
m_stations = []
for i in metro_station:
  m_stations.append(i.get_text())

df['metro'] = [m_stations]


In [0]:
m_stations

In [0]:
# остальная инфа 
apartments_info = card_page.find_all('div', class_="a10a3f92e9--info-text--2uhvD")
apartments_area = apartments_info[0].get_text()
living_apartmetn_area = apartments_info[1].get_text()
kitche_area = apartments_info[2].get_text()
apartments_level = apartments_info[3].get_text()
year_of_construction = apartments_info[4].get_text()
apartsments_text_subscription = card_page.find('p', class_="a10a3f92e9--description-text--1_Lup").get_text()
df['apartments_area'] = [apartments_area]
df['living_apartmetn_area'] = [living_apartmetn_area]
df['kitche_area'] = [kitche_area]
df['apartments_level'] = [apartments_level]
df['year_of_construction'] = [year_of_construction]
df['apartsments_text_subscription'] = [apartsments_text_subscription]

In [0]:
apartsments_text_subscription

In [0]:
additional_appartments_info = card_page.find('ul', class_="a10a3f92e9--container--L-EIV")
appartments_features = []
for i in additional_appartments_info:
  appartments_features.append(i.get_text())

df['appartments_features'] = [appartments_features]


In [0]:
# всякие штуки типо тип квартиры, площадь, какой санузел, тип ремонта
additional_features = []
for i in card_page.find_all(class_="a10a3f92e9--value--3Ftu5"):
  additional_features.append(i.get_text())

df['additional_features'] = [additional_features]

In [0]:
# цена аренды
rent_price = card_page.find(class_="a10a3f92e9--price_value--1iPpd").get_text().replace(u'\xa0', ' ')

In [0]:
df['rent_price'] = [rent_price]

In [0]:
rent_price_info = card_page.find(class_="a10a3f92e9--more_price_rent---5hwY").get_text().replace(u'\xa0', ' ')

In [0]:
df['rent_price_info'] = rent_price_info

In [0]:
additional_payment_info = card_page.find(class_="a10a3f92e9--description--2xRVn").get_text().replace(u'\xa0', ' ').split(",")

In [0]:
df['additional_payment_info'] = [additional_payment_info]

In [0]:
# телефоны
phones = card_page.find(class_="a10a3f92e9--container--35iPF").get_text().replace('Показать телефон', '').replace('Написать сообщение', '').split('+')

In [0]:
df['phones'] = [phones]

In [0]:
# название агентства
agency_name = card_page.find('h2', class_="a10a3f92e9--title--2gUWg").get_text()

In [0]:
df['agency_name'] = [agency_name]

In [0]:
import datetime

In [0]:
upload_d_t = card_page.find(class_="a10a3f92e9--container--3nJ0d").get_text()
view_statistics = card_page.find(class_="a10a3f92e9--link--1t8n1 a10a3f92e9--link--2mJJk").get_text()
if "сегодня" in upload_d_t:
  date = str(datetime.datetime.today())
elif 'вчера' in upload_d_t:
  date = str(datetime.datetime.today())
upload_d_t = [date, upload_d_t]
df['upload_d_t'] = [upload_d_t]